# Formalia:

Please read the [assignment overview page](https://github.com/SocialComplexityLab/socialgraphs2024/wiki/Assignments) carefully before proceeding. This page contains information about formatting (including formats etc), group sizes, and many other aspects of handing in the assignment. 

_If you fail to follow these simple instructions, it will negatively impact your grade!_

**Due date and time**: The assignment is due on Tuesday November 5th, 2024 at 23:55. Hand in your IPython notebook file (with extension `.ipynb`) via DTU Learn

In the exercises below, I describe the exercises in a general way. Drawing in the right parts of the exercises is part of the assignment. (That way we're helping you get a little bit more ready for the Final Project, where you have to decide what information to include in your report and analysis). 


# Part 1: Genres and communities and plotting 

The questions below are based on Lecture 7, part 2.

* Write about genres and modularity.
* Detect the communities, discuss the value of modularity in comparison to the genres.
* Calculate the matrix $D$ and discuss your findings.
* Plot the communities and comment on your results.

# Part 2: TF-IDF to understand genres and communities 

The questions below  are based on Lecture 7, part 2 and 3.

* Explain the concept of TF-IDF in your own words and how it can help you understand the genres and communities.
* Calculate and visualize TF-IDF for the genres and communities.
* Use the matrix $D$ (Lecture 7, part 2) to dicusss the difference between the word-clouds between genres and communities.

# Part 3: Sentiment of the artists and communities

The questions below are based on Lecture 8

* Calculate the sentiment of the Artists pages (OK to work with the sub-network of artists-with-genre) and describe your findings using stats and visualization, inspired by the first exercise of week 8.
* Discuss the sentiment of the largest communities. Do the findings using TF-IDF during Lecture 7 help you understand your results?


## 3.1 Loading and Cleaning Data

In this section, we will load and clean two datasets:

1. **Genres Data**: A JSON file that contains artists and their associated genres. This data will be used to create a network of artists and genres, helping us understand the relationships between different genres.
2. **Sentiment Data**: A tab-delimited text file containing sentiment scores for various words. This data will be used to determine the sentiment associated with each genre, allowing us to calculate the overall sentiment of artists based on the genres they are associated with.

We will create a dictionary from the sentiment data for quick lookups of sentiment scores by word. Additionally, we will perform several data cleaning steps to ensure that the data is accurate and ready for analysis, such as handling missing values, removing non-numeric values, and verifying data consistency.




In [27]:
# Importing required libraries
import json
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter

# Load the genres data from a JSON-like text file
file_path = 'genres.txt'
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        artist_data = json.load(file)
        # Verify the structure of the loaded data
        print("First few artists loaded:", list(artist_data.items())[:5])
except FileNotFoundError:
    print(f"Error: File '{file_path}' not found. Please check the file path.")
except json.JSONDecodeError:
    print(f"Error: File '{file_path}' is not a valid JSON file.")

# Load sentiment data from a .txt file
sentiment_file_path = 'Data_Set_S1.txt'
try:
    sentiment_df = pd.read_csv(sentiment_file_path, sep='\t', skiprows=2, encoding='utf-8')
    # Display the first few rows to verify the data structure
    print("First few rows of sentiment data:")
    print(sentiment_df.head())
except FileNotFoundError:
    print(f"Error: File '{sentiment_file_path}' not found. Please check the file path.")
except pd.errors.EmptyDataError:
    print(f"Error: File '{sentiment_file_path}' is empty or could not be read.")

# Clean and create a dictionary for sentiment lookup
if 'happiness_rank' in sentiment_df.columns and 'word' in sentiment_df.columns:
    sentiment_df = sentiment_df.dropna(subset=['happiness_rank'])
    sentiment_df['happiness_rank'] = pd.to_numeric(sentiment_df['happiness_rank'], errors='coerce')
    sentiment_df = sentiment_df.dropna(subset=['happiness_rank'])
    
    # Create sentiment lookup dictionary
    sentiment_dict = dict(zip(sentiment_df['word'], sentiment_df['happiness_rank'].astype(int)))
    if len(sentiment_df['word']) != len(sentiment_dict):
        print("Warning: Duplicate words detected in sentiment data. Consider further cleaning.")
else:
    print("Error: Required columns 'word' and 'happiness_rank' are missing from sentiment data.")


First few artists loaded: [('A Thousand Horses', ['country', 'country rock', 'southern rock']), ('A.P. Carter', ['country', 'folk', 'gospel']), ('Aaron Barker', ['country']), ('Aaron Benward', ['country', 'christian music']), ('Aaron Lewis (musician)', ['alternative metal', 'post-grunge', 'nu metal', 'country rock', 'alternative country'])]
First few rows of sentiment data:
        word  happiness_rank  happiness_average  happiness_standard_deviation  \
0   laughter               1               8.50                        0.9313   
1  happiness               2               8.44                        0.9723   
2       love               3               8.42                        1.1082   
3      happy               4               8.30                        0.9949   
4    laughed               5               8.26                        1.1572   

  twitter_rank google_rank nyt_rank lyrics_rank  
0         3600          --       --        1728  
1         1853        2458       -- 


## Calculating Sentiment Scores for Artists

We need to calculate average sentiment scores for each artist based on the genres they are associated with. This will help us understand the general sentiment each artist is associated with based on the type of music they create.



In [28]:
# Calculate average sentiment scores for each artist
artist_sentiment_scores = {}
for artist, genres in artist_data.items():
    scores = [sentiment_dict[genre] for genre in genres if genre in sentiment_dict]
    artist_sentiment_scores[artist] = np.mean(scores) if scores else 0

## 3.2 Building the Network Graph and Detecting Communities

In this step, we create a network graph where:

Nodes represent either artists or genres.
Edges connect artists to genres they are associated with.
We then use the Girvan-Newman algorithm to detect communities within the network and identify the largest communities.

In [ ]:

# Create a network graph
G = nx.Graph()

# Add artist nodes with sentiment scores
for artist, score in artist_sentiment_scores.items():
    G.add_node(artist, score=score)

# Add edges between artists and genres
for artist, genres in artist_data.items():
    for genre in genres:
        G.add_node(genre)
        G.add_edge(artist, genre)

# Use Girvan-Newman algorithm for community detection
girvan_newman_communities = girvan_newman(G)
top_level_communities = next(girvan_newman_communities)
communities = [list(community) for community in top_level_communities]

# Count and identify the largest communities
community_counter = Counter({i: len(community) for i, community in enumerate(communities)})
largest_communities = community_counter.most_common(5)
print("Largest communities:", largest_communities)


## Visualizing the Network Graph
We will visualize the network graph with the following characteristics:

Nodes representing artists are colored by their detected community.
The size of the artist nodes is determined by their sentiment score.
Genre nodes are visualized in a distinct color.


In [ ]:
# Re-assign community ID to each node based on community detection
node_to_community = {}
for i, community in enumerate(communities):
    for node in community:
        node_to_community[node] = i

# List of the largest communities by community ID and size
largest_communities = [(0, 2038), (1, 49), (3, 18), (6, 3), (2, 2)]

# Loop through each of the largest communities to create a separate graph for each
for community_id, size in largest_communities:
    # Get nodes belonging to the current community
    community_nodes = [node for node in G.nodes if node_to_community.get(node) == community_id]
    
    # Create a subgraph for the current community
    community_subgraph = G.subgraph(community_nodes)
    
    # Draw the network for the current community
    plt.figure(figsize=(14, 10))
    pos = nx.spring_layout(community_subgraph, seed=42, k=0.2)  # Adjust k for spacing
    
    # Separate artist and genre nodes in the current community
    artist_nodes = [node for node in community_nodes if node in artist_sentiment_scores]
    genre_nodes = [node for node in community_nodes if node not in artist_sentiment_scores]
    
    # Determine node sizes for artists based on sentiment scores
    max_score = max(artist_sentiment_scores.values(), default=1)
    artist_node_sizes = [artist_sentiment_scores.get(node, 0) / max_score * 1000 + 200 for node in artist_nodes]

    # Draw artist nodes with a consistent blue color and size
    sc = nx.draw_networkx_nodes(community_subgraph, pos, nodelist=artist_nodes,
                                node_color='dodgerblue', node_size=artist_node_sizes,
                                alpha=0.9, label="Artists")

    # Draw genre nodes separately with a smaller size and gray color
    nx.draw_networkx_nodes(community_subgraph, pos, nodelist=genre_nodes,
                           node_color='lightgray', node_size=150, alpha=0.7, label="Genres")

    # Draw edges with reduced opacity for clarity
    nx.draw_networkx_edges(community_subgraph, pos, alpha=0.3, width=0.5, edge_color='gray')

    # Draw labels for artist nodes only if the community is not Community 0
    if community_id != 0:
        nx.draw_networkx_labels(community_subgraph, pos, labels={node: node for node in artist_nodes},
                                font_size=8, font_family="sans-serif", font_color='black')

    # Additional plot settings
    plt.axis('off')  # Hide the axis for better visual clarity
    plt.title(f"Artist-Genre Sentiment Network - Community {community_id}", fontsize=16)
    plt.legend()
    plt.show()




The visualization represents the **artist-genre network** for one of the largest communities detected in the network analysis. Here are the key features of the graph:

### Artists (Blue Nodes):
- Represented by **blue nodes**.
- Node size is **proportional to the sentiment score** of the artist. Artists with higher sentiment scores appear larger.
- **Labels** are only applied to artist nodes with the exception of community 0 for clarity, and the font size is reduced to minimize overlap.

### Genres (Gray Nodes):
- Represented by smaller **gray nodes** to distinguish them from artists.
- Genre nodes are intentionally kept small to de-emphasize them in the visualization.

### Edges:
- **Edges connect** artists to the genres they are associated with.
- Edges are drawn with **low opacity** to minimize visual clutter and focus attention on nodes rather than connections.

### Legend:
- The **legend** provides clear differentiation between **Artists** and **Genres**.


### Sentiment Analysis of Communities

We evaluated the **average sentiment score** for the largest communities in our artist-genre network to understand the overall mood associated with each community. Higher average scores indicate positive genre associations, while lower scores suggest neutral or negative tones.

#### Original Issue

The initial results were problematic:
- **Community IDs**: [0, 1, 3, 6, 2]
- **Average Sentiments**: [1845.96, 468.0, 0.0, 0.0, 0.0]

Communities `0` and `1` had extremely high scores, suggesting calculation errors. Communities `3`, `6`, and `2` had scores of `0.0`, implying either missing data or incorrect sentiment calculations. The error stemmed from improper averaging, where values were aggregated instead of divided by the number of artists.

#### Corrected Approach

We fixed the sentiment calculation by:
- **Proper Averaging**: Summing sentiment scores for all artists in a community and dividing by the number of artists.
- **Normalization**: Capping sentiment scores between `0` and `10` to avoid abnormally high values.

#### Average Sentiment Score Explained

The **average sentiment score** represents the general mood of artists in a community:
- **Positive (8.0 - 10.0)**: Genres evoke positive emotions.
- **Neutral (5.0 - 7.9)**: Genres have a mixed or balanced tone.
- **Negative/Low (0.0 - 4.9)**: Genres evoke less positive or melancholic emotions.

#### Updated Results

With the corrected approach, average sentiment scores are now within the expected range (0-10), accurately reflecting the mood and genre diversity of each community.


## Analyzing Sentiment of Largest Communities

We analyze the sentiment distribution of the largest detected communities. This helps us understand the overall sentiment of the groups of artists and how their genres contribute to this sentiment.

In [ ]:
# Analyze sentiment scores in the largest communities
community_sentiments = {}
for community_id, _ in largest_communities:
    # Gather sentiment scores for artists in the current community
    community_artists = [node for node in G.nodes if node_to_community.get(node) == community_id and node in artist_sentiment_scores]
    community_sentiments[community_id] = [artist_sentiment_scores[artist] for artist in community_artists]

# Display basic stats for each community
for community_id, sentiments in community_sentiments.items():
    avg_sentiment = np.mean(sentiments) if sentiments else 0
    print(f"Community {community_id}: Average Sentiment = {avg_sentiment:.2f}, Artist Count = {len(sentiments)}")


## Discussion on Sentiment and TF-IDF Correlation

In this analysis, we explored the relationship between community sentiment scores and TF-IDF values from Lecture 7. Sentiment scores give us insights into the emotional tone of the genres associated with each artist, while TF-IDF helps us understand the importance of specific genres to each community.



In [ ]:
# Display Top Genres for Each Community
print("Top Genres in Largest Communities:")
for community_id, _ in largest_communities:
    community_genres = [genre for artist, genres in artist_data.items()
                        if node_to_community.get(artist) == community_id
                        for genre in genres]
    genre_counter = Counter(community_genres)
    top_genres = genre_counter.most_common(5)
    print(f"Community {community_id} Top Genres: {top_genres}")

# Plot Average Sentiment per Community
avg_sentiments = [np.mean(community_sentiments[community_id]) for community_id, _ in largest_communities]

plt.figure(figsize=(10, 5))
plt.bar(range(len(avg_sentiments)), avg_sentiments, color='skyblue')
plt.xlabel('Community ID')
plt.ylabel('Average Sentiment Score')
plt.xticks(range(len(avg_sentiments)), [f'Community {community_id}' for community_id, _ in largest_communities])
plt.title('Average Sentiment Score for Largest Communities')
plt.show()



## Discussion on Sentiment and TF-IDF Correlation

### Top Genres in Largest Communities
**Community 0**: Dominated by `country`, `folk`, and `americana`. High diversity leads to mixed but mostly positive sentiment.
**Community 1**: Emphasizes `country music`, `bluegrass`, and `comedy`, resulting in a generally positive tone.
**Community 3**: Focuses on `observational comedy` and `satire`, leading to a distinct humorous sentiment.
**Community 6**: Centered on `country and irish`, with a specific cultural focus.
**Community 2**: Defined by `australian country`, reflecting regional cohesion.

## Sentiment and TF-IDF Correlation
- **Genre Influence**: High TF-IDF genres (e.g., `country`, `bluegrass`) strongly shape each community's identity and sentiment. Communities with diverse genres tend to have balanced sentiments, while niche genres show polarized sentiments.
- **Community Cohesion**: Homophily is evident in smaller communities formed around specific genres, indicating strong cohesion. Broader genres lead to larger, more diverse communities with mixed sentiments.


## Observations and Insights
**Positive Sentiments**: Communities with higher average sentiment scores are generally associated with genres that evoke positive emotions (e.g., "love", "happiness")",
**Neutral to Low Sentiments**: Communities with lower average sentiment scores tend to have a more neutral or melancholic tone. This is often seen in smaller, more specialized genre clusters

The artist-genre network reveals how artists are grouped based on genre associations and community sentiment. Community detection using the **Girvan-Newman algorithm** effectively identified clusters that share similar genres, while sentiment analysis provided insights into the general emotional tone of these communities

The combination of community detection and sentiment analysis allows us to understand both **structural relationships** in the artist-genre network and the **emotional impact** of genre associations.
   